In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path = '/content/drive/MyDrive/Email-2-FAQ/data/'
path = "/content/drive/MyDrive/Email2faq/Email-2-FAQ/data/"

In [ ]:
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for cohere: filename=cohere-3.1.5-cp38-cp38-linux_x86_64.whl size=11868 sha256=e18896c126f65bde992ba055b8f4cb5c2f1e26003ccd4907a97613b85381afde
  Stored in directory: /root/.cache/pip/wheels/94/ea/0d/9bc6c2bda325a6ba882384a54adad4f639346dade934adb67b
Successfully built cohere


In [ ]:
import numpy as np 
import pandas as pd

import re
import email
# import cohere

In [ ]:
# Read the data into a DataFrame
emails_df = pd.read_csv(path + 'emails.csv')
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

def clean_column(data):
    if data is not None:
#         stopwords_list = stopwords.words('english')
        #exclusions = ['RE:', 'Re:', 're:']
        #exclusions = '|'.join(exclusions)
        data =  data.lower()
        data = re.sub('re:', '', data)
        data = re.sub('-', '', data)
        data = re.sub('_', '', data)
        # Remove data between square brackets
        data =re.sub('\[[^]]*\]', '', data)
        # removes punctuation
#         data = re.sub(r'[^\w\s]','',data)
        data = re.sub(r'\n',' ',data)
        data = re.sub(r'\t',' ',data)
        data = re.sub(r'[0-9]+','',data)
        # strip html 
        p = re.compile(r'<.*?>')
        data = re.sub(r"\'ve", " have ", data)
        data = re.sub(r"can't", "cannot ", data)
        data = re.sub(r"n't", " not ", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r" m ", " am ", data)
        data = re.sub(r"\'re", " are ", data)
        data = re.sub(r"\'d", " would ", data)
        data = re.sub(r"\'ll", " will ", data)
        data = re.sub('forwarded by phillip k allenhouect on    pm', '',data)
        data = re.sub(r"httpitcappscorpenroncomsrrsauthemaillinkaspidpage", "", data)
        
        data = p.sub('', data)
        # if 'forwarded by:' in data:
        if 'forwarded by ' in data:
            data = data.split('forwarded by ')[0]
        if ('to:' in data) and ("subject:" in data):
            data = data.split('to:')[0]
        data = data.strip()
        return data
    return 'No Subject'

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

# emails_df['Subject_new'] = df['Subject'].apply(clean_column)
emails_df['content_clean'] = emails_df['content'].apply(clean_column)

emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,content_clean
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p,here is our forecast
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p,"randy, can you send me a schedule of the sal..."
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,let's shoot for tuesday at :.


In [ ]:
list_emails = emails_df['content_clean'].sample(10000).reset_index()

In [ ]:
emails_df['content'][76550]

'On your last point, I think that we need to as rate class specific as \npossible without delaying their work.  Disagreements?\n\nJim\n\n\n\n\n\n\tRobert Neustaedter@ENRON_DEVELOPMENT\n\t03/01/2001 06:20 PM\n\t\t \n\t\t To: James D Steffes/NA/Enron@Enron, Harry Kingerski/NA/Enron@Enron, Jeff \nDasovich/NA/Enron@ENRON\n\t\t cc: Tamara Johnson/HOU/EES@EES\n\t\t Subject: MRW Consultant - Status\n\nSpoke with Bill Munson and Roger Yang of MRW\n\nScope of project would be rate forecast for PGE and SCE industrial/commercial \nclass customers for 24-36 month period providing alternate scenarios for \ncertain issues - MRW will provide assumptions Friday morning.\n\nCost - Close of Monday deliverable - $10k\n          - Thursday - $7k\n\nFYI - Submitted RCR for $25K\n\nWith respect to determination of "Net Short" generation cost - MRW suggested \ngeneration cost be allocated on a cost incurrence basis (no subsidies) and \nthat we agree as to mix of DRW portfolio and utilize Enron forward curves

In [ ]:
emails_df['content_clean'][76550]

'on your last point, i think that we need to as rate class specific as  possible without delaying their work.  disagreements?  jim      \trobert neustaedter@enrondevelopment \t// : pm'

In [ ]:
list_emails.to_csv("enron_clean_10K.csv")

In [ ]:
# api_key = 'Gub6qgAUH3VtJUm2HnL2BztpOs06wv73P8Yxinlz'
# co = cohere.Client(api_key)

In [ ]:
# def prompt_test(input_text):
#   prompt = f"""Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn't the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to "the dusty section of the dictionary" to find its latest words.

# TLDR: Wordle has not gotten more difficult to solve.
# --
# Passage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C funding led by ARG Global, with participation from D9 Capital Group and Boulder Capital. Earlier backers also joined the round, including Hilton Group, Roxanne Capital, Paved Roads Ventures, Brook Partners, and Plato Capital.

# TLDR: ArtificialIvan has raised $190 million in Series C funding.
# --
# Passage: {input_text}

# TLDR:"""
#   return prompt

In [ ]:
# n_generations = 1

# print(list_emails['content_clean'][1])
# print()

# ans = []
# for email in [list_emails['content_clean'][1]]:
#     print(prompt_test(email))
#     response = co.generate(
#         model='xlarge',
#         prompt= prompt_test(email),
#         return_likelihoods = 'GENERATION',
#         stop_sequences=["--"],
#         max_tokens=50,
#         temperature=0.7,
#         num_generations=n_generations,
#         k=0,
#         p=0.75
#     )
    
#     summary = response.generations[0].text
#     # gens = []
#     # likelihoods = []
#     # for gen in prediction.generations:
#     #     gens.append(gen.text)
#     #     sum_likelihood = 0
#     #     for t in gen.token_likelihoods:
#     #         sum_likelihood += t.likelihood
#     #     # Get sum of likelihoods
#     #     likelihoods.append(sum_likelihood)
        
#     # df = pd.DataFrame({'generation':gens, 'likelihood': likelihoods})
#     # # Drop duplicates
#     # df = df.drop_duplicates(subset=['generation'])
#     # # Sort by highest sum likelihood
#     # df = df.sort_values('likelihood', ascending=False, ignore_index=True)
    
# #     ans.append(df['generation'][0])
# # ans
# summary

tom  here is the  plan we worked on prior to the bankruptcy filing.  also, the two logistics employees you mentioned are included with bob hall on the file david sent you earlier.      laynie x

Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn't the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to "the dusty section of the dictionary" to find its latest words.

TLDR: Wordle has not gotten more difficult to solve.
--
Passage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C 

' An email from late 2021 discussing layoffs.\n--'

tom  here is the  plan we worked on prior to the bankruptcy filing.  also, the two logistics employees you mentioned are included with bob hall on the file david sent you earlier.      laynie x

Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn't the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to "the dusty section of the dictionary" to find its latest words.

TLDR: Wordle has not gotten more difficult to solve.
--
Passage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C funding led by ARG Global, with participation from D9 Capital Group and Boulder Capital. Earlier backers also joined the round, including Hilton Group, Roxanne Capital, Paved Roads Ventures, Brook Partners, and Plato Capital.

TLDR: ArtificialIvan has raised $190 million in Series C funding.
--
Passage: tom  here is the  plan we worked on prior to the bankruptcy filing.  also, the two logistics employees you mentioned are included with bob hall on the file david sent you earlier.      laynie x

TLDR:
 An email from late 2021 discussing layoffs.\n--

 => exaamples such as these prompt us to not use summarizer for data interpretation

In [ ]:
emails_df['content_clean'][1]

"traveling to have a business meeting takes the fun out of the trip.  especially if you have to prepare a presentation.  i would suggest holding the business plan meetings here then take a trip without any formal business meetings.  i would even try and get some honest opinions on whether a trip is even desired or necessary.  as far as the business meetings, i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  too often the presenter speaks and the others are quiet just waiting for their turn.   the meetings might be better if held in a round table discussion format.    my suggestion for where to go is austin.  play golf and rent a ski boat and jet ski's.  flying somewhere takes too much time."

#testing of bert fine tuned 

In [ ]:
q_s_df = pd.read_csv(path + 'questions_vs_statements_v1.0.csv')
print(q_s_df.shape)
q_s_df.head()

(211515, 3)


,doc,label,target
0,a CBC or Radio-Canada television station loca...,statement,0
1,"Unsurprisingly, these officers enforced socia...",statement,0
2,"In 1952, Thomas Watson, Sr. In what year did I...",question,1
3,What Roman battle took place in the year 446 BC,question,1
4,What often lacks in software developed when it...,question,1


In [ ]:
!pip install transformers # intsall the huggingface library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/question-vs-statement-classifier")

model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/question-vs-statement-classifier")

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from keras_preprocessing.sequence import pad_sequences

from scipy.spatial.distance import pdist,squareform
from sklearn.decomposition import PCA

import torch
import transformers as ppb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
     

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
# from model import BERTClassifier
# from config import BertOptimConfig
# from train import train_model
# from data_loader import QuestionsDataLoader

import pandas as pd
import torch
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from tqdm import tqdm
import logging

## setting the threshold of logger to INFO
logging.basicConfig(filename='data_loader.log', level=logging.INFO)

## creating an object
logger = logging.getLogger() 

In [ ]:
class QuestionsData:
    def __init__(self, start_offset=0, end_offset=1000, max_sequence_length=512):
        """
        Load dataset and custom tokenizer
        """
        ## load data into memory
        self.test_df = q_s_df[start_offset:end_offset]
        ## set max sequence length for model
        self.max_sequence_length = max_sequence_length
        ## get bert tokenizer
        # self.tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-mini', do_lower_case=True)
        self.tokenizer = tokenizer
    
    def train_val_test_split(self):
        """
        Separate out labels and texts
        """
        test_texts = self.test_df['doc'].values
        test_labels = self.test_df.target.values
        
        return test_texts, test_labels
    
    def preprocess(self, texts):
        """
        Add bert token (CLS and SEP) tokens to each sequence pre-tokenization
        """
        ## separate labels and texts before preprocessing
        # Adding CLS and SEP tokens at the beginning and end of each sequence for BERT
        texts_processed = ["[CLS] " + str(sequence) + " [SEP]" for sequence in texts]
        return texts_processed
        
    def tokenize(self, texts):
        """
        Use bert tokenizer to tokenize each sequence and post-process 
        by padding or truncating to a fixed length
        """
        ## tokenize sequence
        tokenized_texts = [self.tokenizer.tokenize(text) for text in (texts)]
        
        ## convert tokens to ids
        text_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in (tokenized_texts)]

        ## pad our text tokens for each sequence
        text_ids_post_processed = pad_sequences(text_ids, 
                                       maxlen=self.max_sequence_length, 
                                       dtype="long", 
                                       truncating="post", 
                                       padding="post") 
        return text_ids_post_processed

    def create_attention_mask(self, text_ids):
        """
        Add attention mask for padding tokens
        """
        attention_masks = []
        # create a mask of 1s for each token followed by 0s for padding
        for seq in (text_ids):
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return attention_masks

    def process_texts(self):
        """
        Apply preprocessing and tokenization pipeline of texts
        """
        ## perform the split
        test_texts, test_labels = self.train_val_test_split()

        ## preprocess train, val, test texts
        test_texts_processed = self.preprocess(test_texts)
        
        del test_texts
        
        ## preprocess train, val, test texts
        test_ids = self.tokenize(test_texts_processed)

       
        del test_texts_processed 
        
        ## create masks for train, val, test texts
        test_masks = self.create_attention_mask(test_ids)
        return (
                test_ids,
                test_masks,
                test_labels
                )

    
    def text_to_tensors(self):
        """
        Converting all the data into torch tensors
        """
        test_ids, \
        test_masks, \
        test_labels = self.process_texts()

        
        ## convert inputs, masks and labels to torch tensors
       
        self.test_inputs = torch.tensor(test_ids)
        self.test_labels = torch.tensor(test_labels)
        self.test_masks = torch.tensor(test_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

class QuestionsDataLoader:

    def __init__(self, data, batch_size=8):
        self.spam_data = data
        self.batch_size = batch_size
        self.create_loaders()
    
    def create_loaders(self):
        """
        Create Torch dataloaders for data splits
        """
        self.spam_data.text_to_tensors()
        
        
        test_data = TensorDataset(self.spam_data.test_inputs, 
                                        self.spam_data.test_masks, 
                                        self.spam_data.test_labels)
        test_sampler = SequentialSampler(test_data)
        self.test_dataloader = DataLoader(test_data, 
                                                sampler=test_sampler, 
                                                batch_size=self.batch_size)

In [ ]:
! pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
# from torch.utils import flat_accuracy

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def eval_model(model, test_dataloader, device):
    ## tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    ## evaluate data for one epoch
    for batch in test_dataloader:
        ## add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        ## unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        ## avoiding model's computation and storage of gradients -> saving memory and speeding up validation
        with torch.no_grad():
            # forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        ## move logits and labels to CPU
        logits = logits['logits'].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    return eval_accuracy/nb_eval_steps

In [ ]:
if __name__=='__main__':
    epochs = 2
    num_labels = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    outer_batch_size = 2000
    accuracy = []
    model.to(device)
    for i in tqdm(range(outer_batch_size, 42000, outer_batch_size)):
        ## batching test set
        data = QuestionsData(start_offset=i-outer_batch_size, end_offset=i)
        quest_loader = QuestionsDataLoader(data)
        quest_loader.create_loaders()
        accuracy.append(eval_model(model=model, test_dataloader=quest_loader.test_dataloader, device=device))

100%|██████████| 20/20 [02:02<00:00,  6.15s/it]


In [ ]:
import numpy as np
np.mean(accuracy)

0.9981500000000001

qc for enron 10K sample

In [ ]:
enron_10K = pd.read_csv("enron_clean_10K.csv")
enron_10K.head()

,Unnamed: 0,index,content_clean
0,0,76783,NaN
1,1,146343,enronoptions announcement you can now access...
2,2,458412,"original message from: thompson, virginia s..."
3,3,317921,dynegy is getting a very nice building. see y...
4,4,468097,"kate per our conversation this afternoon, i ha..."


In [ ]:
enron_10K["content_clean"][2]

'original message from:  thompson, virginia   sent: friday, may ,  : pm'

In [ ]:
enron_10K.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9367 entries, 1 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     9367 non-null   int64 
 1   index          9367 non-null   int64 
 2   content_clean  9367 non-null   object
dtypes: int64(2), object(1)
memory usage: 292.7+ KB


In [ ]:
enron_10K = enron_10K.dropna()

In [ ]:
import re

sentences = []
def split_sentence(text):
  sentences_list = re.split(r"[.;!?]", text)
  sentences.extend(sentences_list)

In [ ]:
split_sentence("original message from:  thompson. virginia   sent: friday, may ,  : pm")

In [ ]:
sentences

['original message from:  thompson', ' virginia   sent: friday, may ,  : pm']

In [ ]:
enron_10K['content_clean'].apply(lambda x: split_sentence(x))

1       None
2       None
3       None
4       None
5       None
        ... 
9994    None
9995    None
9996    None
9997    None
9999    None
Name: content_clean, Length: 9367, dtype: object

In [ ]:
len(sentences)

138628

In [ ]:
sentences_df = pd.DataFrame(sentences)
sentences_df.head()

,0
0,enronoptions announcement you can now access...
1,the web site contains some highlights of the...
2,access enronoptions =) your stock option pro...
3,enron
4,com or by typing the address into your web bro...


In [ ]:
sentences_df.to_csv("sentences_df_enron10k.csv")

In [ ]:
sentences_df = pd.read_csv("sentences_df_enron10k.csv")

In [ ]:
sentences_df.columns = ['id', 'prompt']

In [ ]:
sentences_df.head()

,id,prompt
0,0,enronoptions announcement you can now access...
1,1,the web site contains some highlights of the...
2,2,access enronoptions =) your stock option pro...
3,3,enron
4,4,com or by typing the address into your web bro...


In [ ]:
sample_sen_df = sentences_df.sample(100)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/question-vs-statement-classifier")
def preprocess(texts):
        """
        Add bert token (CLS and SEP) tokens to each sequence pre-tokenization
        """
        ## separate labels and texts before preprocessing
        # Adding CLS and SEP tokens at the beginning and end of each sequence for BERT
        texts_processed = ["[CLS] " + str(sequence) + " [SEP]" for sequence in texts]
        return texts_processed

def tokenize(texts):
        """
        Use bert tokenizer to tokenize each sequence and post-process 
        by padding or truncating to a fixed length
        """
        print(type(texts))
        ## tokenize sequence
        tokenized_texts = [tokenizer.tokenize(text) for text in (te,xts)]
        
        ## convert tokens to ids
        text_ids = [tokenizer.convert_tokens_to_ids(x) for x in (tokenized_texts)]

        ## pad our text tokens for each sequence
        text_ids_post_processed = pad_sequences(text_ids, 
                                       maxlen=512, 
                                       dtype="long", 
                                       truncating="post", 
                                       padding="post") 
        return text_ids_post_processed

def create_attention_mask(text_ids):
        """
        Add attention mask for padding tokens
        """
        attention_masks = []
        # create a mask of 1s for each token followed by 0s for padding
        for seq in (text_ids):
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return attention_masks

sample_sen_df['processed'] = sample_sen_df['prompt'].apply(lambda x: tokenize(preprocess(x)))
sample_sen_df['masks'] = sample_sen_df['processed'].apply(lambda x: create_attention_mask(x))

<class 'list'>


TypeError: ignored

In [ ]:
class Data:
  def text_to_tensors(self):
        """
        Converting all the data into torch tensors
        """
        test_ids = sample_sen_df['processed']
        test_masks = sample_sen_df['masks']

        
        ## convert inputs, masks and labels to torch tensors
        print(sample_sen_df['processed'][0])
        self.test_inputs = torch.tensor(np.array(test_ids).reshape(-1,1))
        self.test_masks = torch.tensor(test_masks)
        self.test_labels = torch.zeros(self.test_masks.size)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

class QuestionsDataLoader:

    def __init__(self, data, batch_size=8):
        self.spam_data = data
        self.batch_size = batch_size
        self.create_loaders()
    
    def create_loaders(self):
        """
        Create Torch dataloaders for data splits
        """
        self.spam_data.text_to_tensors()
        
        
        test_data = TensorDataset(self.spam_data.test_inputs, 
                                        self.spam_data.test_masks, 
                                        self.spam_data.test_labels)
        test_sampler = SequentialSampler(test_data)
        self.test_dataloader = DataLoader(test_data, 
                                                sampler=test_sampler, 
                                                batch_size=self.batch_size)

In [ ]:
import torch
# from torch.utils import flat_accuracy

def eval_model(model, test_dataloader, device, preds):
    ## tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    ## evaluate data for one epoch
    for batch in test_dataloader:
        ## add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        ## unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        ## avoiding model's computation and storage of gradients -> saving memory and speeding up validation
        with torch.no_grad():
            # forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        ## move logits and labels to CPU
        logits = logits['logits'].detach().cpu().numpy()
        pred_flat = np.argmax(logits, axis=1).flatten()
        preds.append(pred_flat)
    return

In [ ]:
if __name__=='__main__':
    epochs = 2
    num_labels = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    outer_batch_size = 2000
    preds = []
    model.to(device)
    ## batching test set
    data = Data()
    quest_loader = QuestionsDataLoader(data)
    quest_loader.create_loaders()
    eval_model(model=model, test_dataloader=quest_loader.test_dataloader, device=device, preds=preds)

KeyError: ignored